In [1]:
import inspect

# Basics

In [2]:
from langchain.prompts.example_selector import (
    LengthBasedExampleSelector,
    SemanticSimilarityExampleSelector,
    MaxMarginalRelevanceExampleSelector,
)

## Example Selector Types

In [3]:
# Drops examples from the bottom of the examples list until the total length of
# the examples plus the query is less than or equal to the max context length
print(inspect.getdoc(LengthBasedExampleSelector))

Select examples based on length.


In [4]:
# Select k examples with the highest semantic similarity to the query
print(inspect.getdoc(SemanticSimilarityExampleSelector))

Example selector that selects examples based on SemanticSimilarity.


In [5]:
# Select k examples based on a combination of semantic similarity and diversity
# It does so by first selecting the example that is most similar to the query, and
# then iteratively adding the remaining examples based on their similarity to the
# query and a penalty for their similarity to the already selected examples
print(inspect.getdoc(MaxMarginalRelevanceExampleSelector))

ExampleSelector that selects examples based on Max Marginal Relevance.

This was shown to improve performance in this paper:
https://arxiv.org/pdf/2211.13892.pdf


In [6]:
# Select k examples based on a combination of semantic similarity and diversity
# It does so by first selecting the example that is most similar to the query, and
# then iteratively adding the remaining examples based on their similarity to the
# query and a penalty for their similarity to the already selected examples
print(inspect.getdoc(MaxMarginalRelevanceExampleSelector))

ExampleSelector that selects examples based on Max Marginal Relevance.

This was shown to improve performance in this paper:
https://arxiv.org/pdf/2211.13892.pdf


# Length Based Example Selector

In [37]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# Prepare standard few shot prompt template
example_template = "Sentence: {sentence}\nEmoji: {emoji}"
example_prompt_template = PromptTemplate.from_template(example_template)
prefix = "You are a chatbot that reacts with emojis to input messages of a user. Please don't use words in your responses, only emojis."
suffix = "Sentence: {sentence}\nEmoji: "
examples = [
    {"sentence": "I love programming.", "emoji": "😀"},
    {"sentence": "Langchain is so powerful.", "emoji": "😎"},
    {"sentence": "I am so happy.", "emoji": "😊"},
    {"sentence": "I am so sad.", "emoji": "😢"},
]

# Instantiate example selector
example_selector = LengthBasedExampleSelector(
    example_prompt=example_prompt_template, 
    examples=examples, 
    max_length=40
)

# Create few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    example_prompt=example_prompt_template,
    example_selector=example_selector,
    prefix=prefix,
    suffix=suffix,
    input_variables=["sentence"],
)

In [38]:
# Create few shot prompt
few_shot_prompt = few_shot_prompt_template.format(sentence="This stuff is amazing!")
print(few_shot_prompt)

You are a chatbot that reacts with emojis to input messages of a user. Please don't use words in your responses, only emojis.

Sentence: I love programming.
Emoji: 😀

Sentence: Langchain is so powerful.
Emoji: 😎

Sentence: I am so happy.
Emoji: 😊

Sentence: I am so sad.
Emoji: 😢

Sentence: This stuff is amazing!
Emoji: 


In [39]:
# Now let's formulate a longer query
few_shot_prompt = few_shot_prompt_template.format(sentence="This sentence is just a litte bit longer than the previous one but is still expressing that this stuff is amazing!")
print(few_shot_prompt)

You are a chatbot that reacts with emojis to input messages of a user. Please don't use words in your responses, only emojis.

Sentence: I love programming.
Emoji: 😀

Sentence: Langchain is so powerful.
Emoji: 😎

Sentence: This sentence is just a litte bit longer than the previous one but is still expressing that this stuff is amazing!
Emoji: 


# Similarity Based Example Selector

In [41]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [56]:
# Instantiate example selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples, 
    vectorstore_cls=FAISS,
    embeddings=OpenAIEmbeddings(),
    k=2
)

# Create few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    example_prompt=example_prompt_template,
    example_selector=example_selector,
    prefix=prefix,
    suffix=suffix,
    input_variables=["sentence"],
)

In [57]:
# Example #1
few_shot_prompt = few_shot_prompt_template.format(sentence="I am very excited!")
print(few_shot_prompt)

You are a chatbot that reacts with emojis to input messages of a user. Please don't use words in your responses, only emojis.

Sentence: I am so happy.
Emoji: 😊

Sentence: I am so sad.
Emoji: 😢

Sentence: I am very excited!
Emoji: 


In [58]:
# Example #2
few_shot_prompt = few_shot_prompt_template.format(sentence="Testing this langchain stuff is so cool!")
print(few_shot_prompt)

You are a chatbot that reacts with emojis to input messages of a user. Please don't use words in your responses, only emojis.

Sentence: Langchain is so powerful.
Emoji: 😎

Sentence: I love programming.
Emoji: 😀

Sentence: Testing this langchain stuff is so cool!
Emoji: 


# Similarity Based Example Selector

In [59]:
# Instantiate example selector
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples, 
    vectorstore_cls=FAISS,
    embeddings=OpenAIEmbeddings(),
    k=2
)

# Create few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    example_prompt=example_prompt_template,
    example_selector=example_selector,
    prefix=prefix,
    suffix=suffix,
    input_variables=["sentence"],
)

In [60]:
# Example #1 --> different from semantic similarity example selector, because it penalizes similarity to already selected examples
few_shot_prompt = few_shot_prompt_template.format(sentence="I am very excited!")
print(few_shot_prompt)

You are a chatbot that reacts with emojis to input messages of a user. Please don't use words in your responses, only emojis.

Sentence: I am so happy.
Emoji: 😊

Sentence: Langchain is so powerful.
Emoji: 😎

Sentence: I am very excited!
Emoji: 
